http://www.skoltech.ru/app/data/uploads/sites/19/2018/03/UDNet_CVPR2018.pdf

In [4]:
DATA_PATH = "../../data"
COMPETITION_FOLDER = "denoise"
FULL_RAW_DATA_PATH = f"{DATA_PATH}/raw/{COMPETITION_FOLDER}"
FULL_FINAL_DATA_PATH = f"{DATA_PATH}/final/{COMPETITION_FOLDER}"

In [5]:
import cv2
import numpy as np
from tqdm import tqdm

In [7]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

# The encoding process
input_img = Input(shape=(1024, 1024, 3))

############
# Encoding #
############

x = Conv2D(filters = 1, kernel_size = (3, 3), activation='relu', padding='same')(input_img)

# Conv1 #
x = Conv2D(filters = 16, kernel_size = (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D(pool_size = (2, 2), padding='same')(x)

# Conv2 #
x = Conv2D(filters = 8, kernel_size = (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size = (2, 2), padding='same')(x) 

# Conv 3 #
x = Conv2D(filters = 8, kernel_size = (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D(pool_size = (2, 2), padding='same')(x)

# Note:
# padding is a hyper-arameter for either 'valid' or 'same'. 
# "valid" means "no padding". 
# "same" results in padding the input such that the output has the same length as the original input.

In [8]:
############
# Decoding #
############

# DeConv1
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)

# DeConv2
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

# Deconv3
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

In [9]:
# Declare the model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [11]:
noisy_img_array = np.load(f'{FULL_FINAL_DATA_PATH}/noisy_img_array.npy', allow_pickle=True)
gt_img_array = np.load(f'{FULL_FINAL_DATA_PATH}/gt_img_array.npy', allow_pickle=True)

In [ ]:
# Train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(noisy_img_array, gt_img_array, test_size=0.2, random_state=42)



## Считаем метрики

In [6]:
# Load the denoised images
noisy_img_array = np.load(f'{FULL_FINAL_DATA_PATH}/noisy_img_array.npy', allow_pickle=True)
gt_img_array = np.load(f'{FULL_FINAL_DATA_PATH}/gt_img_array.npy', allow_pickle=True)
denoised_img_array = np.load(f'{FULL_FINAL_DATA_PATH}/denoised_nn_img_array.npy', allow_pickle=True)

# Plot the denoised images side by side with the noisy and ground truth images
import matplotlib.pyplot as plt
import random

def plot_denoised_images(noisy_img_array, denoised_img_array, gt_img_array, num_images=5):
    fig, axes = plt.subplots(num_images, 3, figsize=(15, 15))
    for i in range(num_images):
        idx = random.randint(0, noisy_img_array.shape[0])
        axes[i, 0].imshow(noisy_img_array[idx])
        axes[i, 0].set_title("Noisy Image")
        axes[i, 1].imshow(denoised_img_array[idx])
        axes[i, 1].set_title("Denoised Image")
        axes[i, 2].imshow(gt_img_array[idx])
        axes[i, 2].set_title("Ground Truth Image")
    plt.show()

plot_denoised_images(noisy_img_array, denoised_img_array, gt_img_array)


FileNotFoundError: [Errno 2] No such file or directory: '../../data/final/denoise/denoised_nn_img_array.npy'

In [ ]:
# Посчитать метрику по PSNR и SSIM

# scikit-image
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Calculate the PSNR and SSIM for the denoised images
psnr_scores = [psnr(gt_img_array[i], denoised_img_array[i]) for i in tqdm(range(len(gt_img_array)))]

# Fix ssim ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.
# Time-consuming...
ssim_scores = [ssim(gt_img_array[i], denoised_img_array[i], multichannel=True, win_size=3) for i in tqdm(range(len(gt_img_array)))]



In [ ]:
def print_perforamnce(what:str, psnr_scores, inference_time_in_seconds, ssim_scores=None):
    print(f"{what} performance")
    print(f"Metrics:")
    print(f"\tPSNR: {np.mean(psnr_scores)}")
    print(f"\tSSIM: {np.mean(ssim_scores)}")
    print(f"Inference performance:")
    print(f"\tTime per image:\t{(inference_time_in_seconds)/len(psnr_scores)} seconds")
    print(f"\tFPS:\t\t{len(psnr_scores)/inference_time_in_seconds} fps")
    print(f"\tTotal time:\t{inference_time_in_seconds} seconds")
    print(f"\tImages count:\t{len(psnr_scores)}")
    print(f"\tGPU ENABLED?:\t{cv2.ocl.useOpenCL()}")
    print(f"="*80)
    print(f"Build info:\t{cv2.getBuildInformation()}")

print_perforamnce("cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)", psnr_scores, inference_time_in_seconds, ssim_scores)

In [ ]:
# Zoom in on a small region of the image to see the difference
def plot_denoised_images(noisy_img_array, denoised_img_array, gt_img_array, num_images=5):
    fig, axes = plt.subplots(num_images, 3, figsize=(15, 15))
    for i in range(num_images):
        idx = random.randint(0, noisy_img_array.shape[0])
        axes[i, 0].imshow(noisy_img_array[idx][100:200, 100:200])
        axes[i, 0].set_title("Noisy Image")
        axes[i, 1].imshow(denoised_img_array[idx][100:200, 100:200])
        axes[i, 1].set_title("Denoised Image")
        axes[i, 2].imshow(gt_img_array[idx][100:200, 100:200])
        axes[i, 2].set_title("Ground Truth Image")
    plt.show()

plot_denoised_images(noisy_img_array, denoised_img_array, gt_img_array)